In [1]:
from requests_html import HTML
import selenium
import datetime as dt
from selenium import webdriver

In [2]:
page = 'https://nha.chotot.com/tp-ho-chi-minh/mua-ban-nha-dat?page='

In [3]:
# Hàm nhận vào link tới trang web.
# Hàm trả ra Element chứa thông tin cần thiết.
# "Vì chotot sẽ có thông tin ưu tiên và cả thông tin bình thường,...
# nên hàm sẽ trả về một tuple chứa thông tin ưu tiên trước, sau đó là thông tin bình thường"
def get_inform_from_page(link, num_page, ff):
    link = link + str(num_page)
    ff.get(link)
    # Bấm tắt thông báo hiện lên
    try:
        ff.find_element_by_css_selector('.ctTooltipClose').click()
    except:
        i = 1
    # Đổi giao diện để có thể thao tác dễ hơn
    ff.find_element_by_css_selector('.glyphicon.glyphicon-th-large').click()
    r = HTML(html=ff.page_source)
    tin_bt = r.find('._3AO2qzWF3LGsnNA73I-RLF')
    # tin_uu_tien = ff.find_element_by_class_name('ctAdLitingThumbnail')
    return tin_bt

In [4]:
# Thuộc tính bao gồm tựa đề, giá tiền, loại hình nhà (cá nhân hay môi giới hay bất động sản), 
# link tới bài viết cụ thể, thời gian đăng tin.
#################################################################################################
# Trong các bài viết cụ thể còn có các thuộc tính khác như:
# - Diện tích
# - Hướng cửa chính
# - Số phòng vệ sinh
# - Số tầng
# - Loại hình nhà ở
# - Giấy tờ pháp lý
# - Đặc điểm nhà đất
# - Địa chỉ cụ thể

In [5]:
title1_list = []
price_list = []
title2_list = []
type1_list = []
time_list = []
link_list = []

In [6]:
link_prefit = 'https://nha.chotot.com/'
ff = webdriver.Firefox()
for ind in range(3487,4500):
    thong_thuong = get_inform_from_page(page, ind, ff)
    n = len(thong_thuong)
    for i in range(n):
        # Lấy link dắt tới thông tin cụ thể
        link = thong_thuong[i].find('a', first=True).attrs['href']
        link = link_prefit + link
        #Price là giá tiền của căn nhà đó.
        tmp = thong_thuong[i].find('p', first=True)
        if tmp == None:
            continue
        price = tmp.text
        # Type là loại hình mua bán, cá nhân, môi giới??
        type1 = thong_thuong[i].find('img')[1].attrs['alt']
        # Time sẽ là thời gian đăng tin, Time ở trên trang web sẽ ghi là 1 ngày trước, 1 phút trước, ...
        # Nên ta sẽ trích thời gian hiện tại và trừ ra khoảng thời gian đó để có được thời gian đăng tin.
        time = dt.datetime.now()
        time_text = thong_thuong[i].find('span')
        time_text = time_text[-1].text
        diff = int(time_text.split(' ')[0])
        if 'phút' in time_text.lower():
            time = time - dt.timedelta(minutes=diff)
        elif 'giờ' in time_text.lower():
            time = time - dt.timedelta(hours=diff)
        elif 'ngày' in time_text.lower():
            time = time - dt.timedelta(days=diff)
        elif 'tháng' in time_text.lower():
            time = time - dt.timedelta(diff*365/12)
        elif 'tuần' in time_text.lower():
            time = time - dt.timedelta(days=diff*7)
    
        price_list.append(price)
        type1_list.append(type1)
        time_list.append(time)
        link_list.append(link)

KeyboardInterrupt: 

In [8]:
print(len(link_list))
import pandas as pd
data = pd.DataFrame({'price': price_list, 'type':type1_list, 'time': time_list, 'link':link_list})
data.to_csv('out-3487-4331.csv',encoding='utf-8', index=None)

16244


In [7]:
ind

4331

In [ ]:
ind